## Kymograph Cluster Code

In [ ]:
import trenchripper as tr

#### Start Dask

In [ ]:
kymo_controller = tr.cluster.dask_controller(
    walltime="04:00:00", local=False, n_workers=20, memory="2GB"
)
kymo_controller.startdask()
kymo_controller.daskcluster.start_workers()

In [ ]:
kymo_controller.daskclient

#### Perform Extraction

In [ ]:
hdf5_extractor = tr.ndextract.hdf5_fov_extractor(
    "/n/scratch2/de64/full_pipeline_test/40x_Ph2_Test_1.5.nd2",
    "/n/scratch2/de64/full_pipeline_test/hdf5",
)  ##find a more elegent way to do grids....

In [ ]:
hdf5_extractor.writemetadata()

In [ ]:
fov_list = list(range(0, 80))
# fov_list = list(range(16)) + list(range(29,40))

In [ ]:
kymo_controller.mapfovs(hdf5_extractor.extract_fov, fov_list)

In [ ]:
# props = kymo_controller.daskclient.gather(kymo_controller.futures) #this will hang until all futures are done

In [ ]:
# hdf5_extractor.extract_fov(0)

#### End Dask, Cancel Jobs

In [ ]:
kymo_controller.shutdown()

In [ ]:
# scancel --name=dask-worker --user=de64

#### Generate Kymographs

#### Start Dask

In [ ]:
kymo_controller = tr.cluster.dask_controller(
    walltime="04:00:00", local=False, n_workers=40, memory="4GB"
)
kymo_controller.startdask()
kymo_controller.daskcluster.start_workers()

In [ ]:
kymo_controller.daskclient

In [ ]:
input_file_prefix = "/n/scratch2/de64/for_sylvia/timelapse/hdf5/fov_"
output_path = "/n/scratch2/de64/for_sylvia/timelapse/kymo"
all_channels = ["channel_Phase"]

kymo = tr.kymograph.kychunker(
    input_file_prefix=input_file_prefix,
    output_path=output_path,
    all_channels=all_channels,
    trench_len_y=110,
    padding_y=20,
    trench_width_x=20,
    t_chunk=25,
    t_range=(0, 400),
    y_percentile=90,
    y_min_edge_dist=50,
    smoothing_kernel_y=(17, 1),
    triangle_nbins=50,
    triangle_scaling=1.2,
    top_orientation=0,
    x_percentile=85,
    background_kernel_x=(301, 1),
    smoothing_kernel_x=(9, 1),
    otsu_nbins=50,
    otsu_scaling=0.7,
)

In [ ]:
hdf5_extractor.metadata["channels"]  ##these names should match my convention

In [ ]:
# fov_list = list(range(15)) + list(range(67,80))
fov_list = list(range(20)) + list(range(60, 80))

In [ ]:
kymo_controller.mapfovs(kymo.generate_kymograph, fov_list)

maybe an estimate of memory usage

In [ ]:
import numpy as np

In [ ]:
kymo.generate_kymograph(15)  ## figure out this error ##

In [ ]:
from matplotlib import pyplot as plt

plt.plot(output[0]["data"][:, 0])

### NOTE

- **How to deal with some fovs having a different number of rows than other fovs??? Try to make more flexible...**

maybe add a kymograph viewer here

### Note
There may be an issue where hdf5 files do not close properly on crashes...

In [ ]:
# props = kymo_controller.daskclient.gather(kymo_controller.futures) #this will hang until all futures are done

#### End Dask, Cancel Jobs

In [ ]:
kymo_controller.shutdown()

In [ ]:
# scancel --name=dask-worker --user=de64

#### Note: access request to the transfer partition pending; automate this bit later

In [ ]:
import subprocess

in_path = "/n/files/SysBio/PAULSSON LAB/Daniel/Image_Data/test"
out_path = "/n/scratch2/de64/for_sylvia/"
wrap_str = "rsync " + in_path + " " + out_path
cmd_str = 'sbatch -p transfer -t 0-12:00 --wrap="' + wrap_str + '"'

subprocess.check_output(cmd_str, shell=True)

In [ ]:
cmd_str

In [ ]:
!sbatch -p transfer -t 0-12:00 --wrap="rsync /n/files/SysBio/PAULSSON\LAB/Daniel/Image_Data/Bacillus_Project/test /n/scratch2/de64/for_sylvia/"

#### Note: need to find a way to associate lanes from init barcoding snapshot with imaging lanes

plan:
- extract and segment (with phase) barcoding file
- extract and segment experiemnt file
- for both, note x positions of t=0 trenches
- compare these values to map trenches to eachother...

In [ ]:
import h5py
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
np.array([1, 2])[:]

In [ ]:
h5py_handle = h5py.File("/n/scratch2/de64/for_sylvia/timelapse/kymo/kymo_10.hdf5", "r")

In [ ]:
list(h5py_handle.keys())

In [ ]:
top = h5py_handle["1/channel_Phase"]

In [ ]:
# top = h5py_handle["channel_Phase"]

In [ ]:
top

In [ ]:
# plt.imshow(top[50,:,:,20])
tr.tplot.plot_kymograph(top[10, :, :, 0:20])

In [ ]:
def test_fn():
    h5py_handle = h5py.File(
        "/n/scratch2/de64/for_sylvia/timelapse/hdf5/fov_0.hdf5", "r"
    )
    top = h5py_handle["channel_Phase"]
    test = top[:, :, 0:50]
    output2 = np.percentile(test, 85, axis=1, interpolation="lower")
    output3 = np.percentile(test, 85, axis=1, interpolation="higher")
    output4 = np.percentile(test, 85, axis=1, interpolation="linear")
    h5py_handle.close()

In [ ]:
%load_ext line_profiler
%lprun -f test_fn test_fn()

In [ ]:
h5py_handle = h5py.File("/n/scratch2/de64/for_sylvia/timelapse/hdf5/fov_0.hdf5", "r")
top = h5py_handle["channel_Phase"]
slc = [slice(None)] * len(top.shape)
slc[2] = slice(0, 50)
print(slc)
test = top[tuple(slc)]
h5py_handle.close()

In [ ]:
h5py_handle = h5py.File("/n/scratch2/de64/for_sylvia/timelapse/hdf5/fov_0.hdf5", "r")
top = h5py_handle["channel_Phase"]
test = top[:, :, range(0, 50)]
h5py_handle.close()

In [ ]:
h5py_handle = h5py.File("/n/scratch2/de64/for_sylvia/timelapse/hdf5/fov_0.hdf5", "r")
top = h5py_handle["channel_Phase"]
test = top[:, :, list(range(0, 50))]
h5py_handle.close()

In [ ]:
h5py_handle = h5py.File("/n/scratch2/de64/for_sylvia/timelapse/hdf5/fov_0.hdf5", "r")
top = h5py_handle["channel_Phase"]
test = np.take(top, list(range(0, 50)), axis=2)
h5py_handle.close()

In [ ]:
np.take(
    h5py_handle["channel_Phase"], [1, 2, 3, 4, 5], axis=2
)  ##this operation is killing me

In [ ]:
h5py_handle["channel_Phase"][:, :, :20].shape

In [ ]:
h5py_handle.close()

In [ ]:
h5py_handle["1/channel_YFP"]

In [ ]:
h5py_handle.close()

In [ ]:
testlist = ["success"]
potato = "fail"

In [ ]:
def test(potato, testlist):
    exec("potato = testlist")
    return potato

In [ ]:
test(potato, ["moo"])

In [ ]:
potato

In [ ]:
h5py_handle = h5py.File("/n/scratch2/de64/for_sylvia/timelapse/hdf5/fov_0.hdf5", "r")

In [ ]:
h5py_handle["channel_Phase"].chunks

In [ ]:
h5py_handle["channel_Phase"]

In [ ]:
target[slc]